Reference: https://huggingface.co/docs/transformers/training

In [1]:
# Transformers installation
!pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 11.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


## Load the model to be finetuned

In [2]:
# load a TensorFlow model
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from datasets import load_dataset

## Prepare a dataset

In [4]:
# function to tokenize the data

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [5]:
# convert dataset to TensorFlow format
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [6]:
import pandas as pd
import datasets
from datasets import Dataset

def sample_equally(tokenized_data, train_size, val_size, test_size):

    sizes = [train_size, val_size, test_size]
    data_types = ['train', 'validation', 'test']

    temp = []
    for data_type, size in zip(data_types, sizes):
        # convert to df and then sample equally
        df = pd.DataFrame(tokenized_data[data_type])
        new_df = df.groupby('label').apply(lambda x: x.sample(n=size)).reset_index(drop = True)
        temp.append(new_df)

    # convert to Dataset now
    data_set = datasets.DatasetDict(
        {"train": Dataset.from_pandas(temp[0]),
         "validation": Dataset.from_pandas(temp[1]),
         "test": Dataset.from_pandas(temp[2])
         })

    return data_set

### IMDB dataset

Train: 3310 each

Validation: 428 each

Test: 909 each





In [7]:
dataset_imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
tokenized_imdb = dataset_imdb.map(tokenize_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [9]:
tokenized_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [10]:
# def sample_equally_noval_set(tokenized_data, train_size, val_size, test_size):

#     train_set = tokenized_data["train"].shuffle(seed=42).select(range(train_size*2))
#     val_set = tokenized_data["train"].shuffle(seed=42).select(range(train_size*2, (train_size+val_size)*2))

#     # df = pd.DataFrame(tokenized_data["train"])

#     # train_df = df.iloc[:train_size*2, :]
#     # val_df = df.iloc[(train_size*2)+(val_size*2):, :]

#     train_df = pd.DataFrame(train_set)
#     val_df = pd.DataFrame(val_set)
#     test_df = pd.DataFrame(tokenized_data['test'])

#     tokenized_data = datasets.DatasetDict(
#         {"train": Dataset.from_pandas(train_df),
#         "validation": Dataset.from_pandas(val_df),
#         "test": Dataset.from_pandas(test_df)
#         })

#     sizes = [train_size, val_size, test_size]
#     data_types = ['train', 'validation', 'test']

#     temp = []
#     for data_type, size in zip(data_types, sizes):
#         # convert to df and then sample equally
#         df = pd.DataFrame(tokenized_data[data_type])
#         new_df = df.groupby('label').apply(lambda x: x.sample(n=size)).reset_index(drop = True)
#         temp.append(new_df)

#     # convert to Dataset now
#     data_set = datasets.DatasetDict(
#         {"train": Dataset.from_pandas(temp[0]),
#             "validation": Dataset.from_pandas(temp[1]),
#             "test": Dataset.from_pandas(temp[2])
#             })

#     return data_set

In [11]:
# tokenized_imdb = sample_equally_noval_set(tokenized_imdb, 3310, 428, 909)
# tokenized_imdb

In [10]:
imdb_train = tokenized_imdb["train"].shuffle(seed=42).select(range(6920))
imdb_val = tokenized_imdb["train"].shuffle(seed=42).select(range(6920, 7792))   #872
imdb_test = tokenized_imdb["test"].shuffle(seed=42).select(range(1821))

In [11]:
imdb_test['label'].count(0)

917

In [12]:
tf_train_dataset_imdb = imdb_train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset_imdb = imdb_val.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset_imdb = imdb_test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

In [23]:
# finetuning

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset_imdb, validation_data=tf_validation_dataset_imdb, epochs=2)

Epoch 1/2
865/865 [==============================] - 892s 1s/step - loss: 0.3342 - sparse_categorical_accuracy: 0.8574 - val_loss: 0.2294 - val_sparse_categorical_accuracy: 0.9117
Epoch 2/2
865/865 [==============================] - 878s 1s/step - loss: 0.1749 - sparse_categorical_accuracy: 0.9370 - val_loss: 0.2741 - val_sparse_categorical_accuracy: 0.9060


In [24]:
model.evaluate(tf_test_dataset_imdb)

228/228 [==============================] - 77s 339ms/step - loss: 0.2450 - sparse_categorical_accuracy: 0.9094


[0.24502183496952057, 0.9093904495239258]

In [25]:
model.evaluate(tf_test_dataset_sst)

228/228 [==============================] - 77s 338ms/step - loss: 3.3083 - sparse_categorical_accuracy: 0.1755


[3.3083295822143555, 0.17546755075454712]

In [26]:
model.evaluate(tf_test_dataset_yelp)

228/228 [==============================] - 77s 339ms/step - loss: 0.3183 - sparse_categorical_accuracy: 0.8759


[0.31827512383461, 0.8758923411369324]

###SST2

In [13]:
# sst-2

dataset_sst = load_dataset("gpt3mix/sst2")
tokenized_sst = dataset_sst.map(tokenize_function, batched=True)
tokenized_sst

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/gpt3mix___sst2/default/0.0.0/90167692658fa4abca2ffa3ede1a43a71e2bf671078c5c275c64c4231d5a62fa. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6920
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [14]:
tokenized_sst = sample_equally(tokenized_sst, 3310, 428, 909)

In [15]:
tokenized_sst

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6620
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 856
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1818
    })
})

In [16]:
sst_train = tokenized_sst["train"].shuffle(seed=42)
sst_val = tokenized_sst["validation"].shuffle(seed=42)
sst_test = tokenized_sst["test"].shuffle(seed=42)

tf_train_dataset_sst = sst_train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset_sst = sst_val.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset_sst = sst_test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
sst_val['label'].count(1)

428

In [ ]:
sst_val['label'].count(0)

428

In [ ]:
sst_train['label'].count(1)

3310

In [ ]:
sst_train['label'].count(0)

3310

In [ ]:
sst_test['label'].count(0)

909

In [ ]:
sst_test['label'].count(1)

909

In [28]:
# train on sst2

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset_sst, validation_data=tf_validation_dataset_sst, epochs=2)

Epoch 1/2
828/828 [==============================] - 826s 979ms/step - loss: 0.3547 - sparse_categorical_accuracy: 0.8517 - val_loss: 0.2816 - val_sparse_categorical_accuracy: 0.8820
Epoch 2/2
828/828 [==============================] - 815s 985ms/step - loss: 0.1535 - sparse_categorical_accuracy: 0.9459 - val_loss: 0.2694 - val_sparse_categorical_accuracy: 0.8972


In [29]:
model.evaluate(tf_test_dataset_sst)

228/228 [==============================] - 76s 333ms/step - loss: 0.2437 - sparse_categorical_accuracy: 0.8982


[0.24372270703315735, 0.8982398509979248]

In [30]:
model.evaluate(tf_test_dataset_imdb)

228/228 [==============================] - 77s 340ms/step - loss: 2.0686 - sparse_categorical_accuracy: 0.2109


[2.0685598850250244, 0.21087314188480377]

In [31]:
model.evaluate(tf_test_dataset_yelp)

228/228 [==============================] - 78s 340ms/step - loss: 2.5467 - sparse_categorical_accuracy: 0.1691


[2.546705961227417, 0.1691378355026245]

### Yelp

In [17]:
dataset_yelp = load_dataset("yelp_polarity")
tokenized_yelp = dataset_yelp.map(tokenize_function, batched=True)
tokenized_yelp

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/560 [00:00<?, ?ba/s]

  0%|          | 0/38 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 38000
    })
})

In [18]:
# tokenized_yelp = sample_equally(tokenized_yelp, 3310, 428, 909)
# tokenized_yelp

In [19]:
yelp_train = tokenized_yelp["train"].shuffle(seed=42).select(range(6920))
yelp_val = tokenized_yelp["train"].shuffle(seed=42).select(range(6920, 7792))
yelp_test = tokenized_yelp["test"].shuffle(seed=42).select(range(1821))

tf_train_dataset_yelp = yelp_train.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset_yelp = yelp_val.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset_yelp = yelp_test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

In [20]:
yelp_train['label'].count(0)

3436

In [ ]:
yelp_train['label'].count(1)

3484

In [ ]:
yelp_test['label'].count(0)

931

In [ ]:
yelp_test['label'].count(1)

890

In [21]:
# train on yelp

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset_yelp, validation_data=tf_validation_dataset_yelp, epochs=2)

Epoch 1/2
865/865 [==============================] - 891s 1s/step - loss: 0.2273 - sparse_categorical_accuracy: 0.9069 - val_loss: 0.1801 - val_sparse_categorical_accuracy: 0.9312
Epoch 2/2
865/865 [==============================] - 867s 1s/step - loss: 0.0960 - sparse_categorical_accuracy: 0.9642 - val_loss: 0.1697 - val_sparse_categorical_accuracy: 0.9404


In [22]:
model.evaluate(tf_test_dataset_yelp)

228/228 [==============================] - 79s 345ms/step - loss: 0.1695 - sparse_categorical_accuracy: 0.9357


[0.16948707401752472, 0.9357495903968811]

In [23]:
model.evaluate(tf_test_dataset_imdb)

228/228 [==============================] - 78s 341ms/step - loss: 0.3354 - sparse_categorical_accuracy: 0.8644


[0.33537477254867554, 0.8643602132797241]

In [24]:
model.evaluate(tf_test_dataset_sst)

228/228 [==============================] - 77s 338ms/step - loss: 2.5367 - sparse_categorical_accuracy: 0.1909


[2.5366580486297607, 0.19086909294128418]